In [11]:
from neutrino.framework.torch_framework import TorchFramework
from neutrino.job import Neutrino
from torchvision import transforms
import torchvision
import torch
framework = TorchFramework()

In [12]:
def get_cifar100_dataset(dataroot, batch_size):
    trainset = torchvision.datasets.CIFAR100(root=dataroot,
                                             train=True,
                                             download=True,
                                             transform=transforms.Compose([
                                                 transforms.RandomCrop(32, padding=4),
                                                 transforms.RandomHorizontalFlip(),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                                      (0.2023, 0.1994, 0.2010))
                                             ]))
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=4, pin_memory=True)

    testset = torchvision.datasets.CIFAR100(root=dataroot,
                                            train=False,
                                            download=True,
                                            transform=transforms.Compose([
                                                transforms.ToTensor(),
                                                transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                                     (0.2023, 0.1994, 0.2010))
                                            ]))
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=False, num_workers=4, pin_memory=True)

    return {
            'train': trainloader,
            'test': testloader
            }

In [6]:
reference_model = torchvision.models.resnet18(pretrained=True)
reference_model

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /mnt/nfshome/risavsingh.saingar/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
data = get_cifar100_dataset("./Data", 16)
config = {
    "optimization": "compression",
    "delta": "1",
    'export': {
        'format': ['onnx'],
        'kwargs': {
            'root_path': "./Models",
            'precision': 'fp32', # ('fp32' or 'fp16'), only for onnx, dlrt formats
            'resolutions': [(32, 32), (36, 36)] # list of tuples, only for onnx, dlrt formats
        }
    }
}
model = Neutrino(framework, data, model=reference_model, config=config)

Files already downloaded and verified
Files already downloaded and verified


In [27]:
model.run()

2023-01-09 08:13:01 - INFO: Starting job with ID: B5DEBCB7
2023-01-09 08:13:01 - INFO: Args: -f, /mnt/nfshome/risavsingh.saingar/.local/share/jupyter/runtime/kernel-927196e7-204f-4f91-9926-b6a75a11c718.json
2023-01-09 08:13:01 - INFO: 
+------------------------------------------------------------------------------------+
| Neutrino 5.3.3                                                 09/01/2023 08:13:01 |
+------------------------------------------------------------------------------------+
| License                                                    HVHEP-GCXFZ-KGUTB-TVGPI |
| Status                                                                       Valid |
| Expires                                                        2024-01-09 07:14:07 |
| Features                                                                           |
+------------------------------------------------------------------------------------+

2023-01-09 08:13:01 - INFO: Neutrino Caching disabled
2023-01-09 08

In [47]:
import onnx
from onnx2torch import convert
m = onnx.load("./second_best_modelfp32_dynamic_shape.onnx")
convert(m)

GraphModule(
  (Conv_0): Conv2d(3, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Relu_0): ReLU()
  (Conv_1): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Relu_1): ReLU()
  (Conv_2): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Add_0): OnnxBinaryMathOperation()
  (Relu_2): ReLU()
  (Conv_3): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Relu_3): ReLU()
  (Conv_4): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Add_1): OnnxBinaryMathOperation()
  (Relu_4): ReLU()
  (Conv_5): Conv2d(64, 128, kernel_size=[3, 3], stride=[2, 2], padding=[1, 1], dilation=[1, 1])
  (Relu_5): ReLU()
  (Conv_6): Conv2d(128, 128, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Conv_7): Conv2d(64, 128, kernel_size=[1, 1], stride=[2, 2], padding=[0, 0], dilation=[1, 1])
  (Add_2): OnnxBinaryMathOperation()

In [48]:
import onnx
from onnx2torch import convert
m = onnx.load("./Models/ref_modelfp32_dynamic_shape.onnx")
convert(m)

GraphModule(
  (Conv_0): Conv2d(3, 64, kernel_size=[7, 7], stride=[2, 2], padding=[3, 3], dilation=[1, 1])
  (Relu_0): ReLU()
  (MaxPool_0): MaxPool2d(kernel_size=[3, 3], stride=[2, 2], padding=[1, 1], dilation=1, ceil_mode=False)
  (Conv_1): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Relu_1): ReLU()
  (Conv_2): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Add_0): OnnxBinaryMathOperation()
  (Relu_2): ReLU()
  (Conv_3): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Relu_3): ReLU()
  (Conv_4): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Add_1): OnnxBinaryMathOperation()
  (Relu_4): ReLU()
  (Conv_5): Conv2d(64, 128, kernel_size=[3, 3], stride=[2, 2], padding=[1, 1], dilation=[1, 1])
  (Relu_5): ReLU()
  (Conv_6): Conv2d(128, 128, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Conv_7): Conv2d(64, 128,

In [34]:
help(m)

Help on ModelProto in module onnx.onnx_ml_pb2 object:

class ModelProto(google.protobuf.pyext._message.CMessage, google.protobuf.message.Message)
 |  A ProtocolMessage
 |  
 |  Method resolution order:
 |      ModelProto
 |      google.protobuf.pyext._message.CMessage
 |      google.protobuf.message.Message
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  doc_string
 |      Field onnx.ModelProto.doc_string
 |  
 |  domain
 |      Field onnx.ModelProto.domain
 |  
 |  functions
 |      Field onnx.ModelProto.functions
 |  
 |  graph
 |      Field onnx.ModelProto.graph
 |  
 |  ir_version
 |      Field onnx.ModelProto.ir_version
 |  
 |  metadata_props
 |      Field onnx.ModelProto.metadata_props
 |  
 |  model_version
 |      Field onnx.ModelProto.model_version
 |  
 |  opset_import
 |      Field onnx.ModelProto.opset_import
 |  
 |  producer_name
 |      Field onnx.ModelProto.producer_name
 |  
 |  producer_version
 |      Field onnx.ModelProto.producer_version
 

In [37]:
m.graph

GraphModule(
  (Conv_0): Conv2d(3, 64, kernel_size=[7, 7], stride=[2, 2], padding=[3, 3], dilation=[1, 1])
  (Relu_0): ReLU()
  (MaxPool_0): MaxPool2d(kernel_size=[3, 3], stride=[2, 2], padding=[1, 1], dilation=1, ceil_mode=False)
  (Conv_1): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Relu_1): ReLU()
  (Conv_2): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Add_0): OnnxBinaryMathOperation()
  (Relu_2): ReLU()
  (Conv_3): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Relu_3): ReLU()
  (Conv_4): Conv2d(64, 64, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Add_1): OnnxBinaryMathOperation()
  (Relu_4): ReLU()
  (Conv_5): Conv2d(64, 128, kernel_size=[3, 3], stride=[2, 2], padding=[1, 1], dilation=[1, 1])
  (Relu_5): ReLU()
  (Conv_6): Conv2d(128, 128, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], dilation=[1, 1])
  (Conv_7): Conv2d(64, 128,

In [46]:
import argparse

from deeplite_torch_zoo import get_data_splits_by_name, get_model_by_name
from neutrino.framework.torch_framework import TorchFramework
from neutrino.job import Neutrino

# parser = argparse.ArgumentParser()
# # model/dataset args
# parser.add_argument('--dataset', metavar='DATASET', default='cifar100', help='dataset to use')
# parser.add_argument('-r', '--data_root', metavar='PATH', default='', help='dataset data root path')
# parser.add_argument('-b', '--batch_size', type=int, metavar='N', default=128, help='mini-batch size')
# parser.add_argument('-j', '--workers', type=int, metavar='N', default=4, help='number of data loading workers')
# parser.add_argument('-a', '--arch', metavar='ARCH', default='resnet18', help='model architecture')

# # neutrino args
# parser.add_argument('-d', '--delta', type=float, metavar='DELTA', default=1, help='accuracy drop tolerance')
# parser.add_argument('-l', '--level', type=int, default=1, help='level', choices=(1, 2))
# parser.add_argument('-o', '--optimization', type=str, default='compression', choices=('compression', 'latency'))
# parser.add_argument('--deepsearch', action='store_true', help="to consume the delta as much as possible")
# parser.add_argument('--fp16', action='store_true', help="export to fp16 as well if it is possible")
# parser.add_argument('--dryrun', action='store_true', help="force all loops to early break")
# parser.add_argument('--horovod', action='store_true', help="activate horovod")
# parser.add_argument('--device', type=str, metavar='DEVICE', default='GPU', help='Device to use, CPU or GPU')
# parser.add_argument('--lr', default=0.1, type=float,
#                     help='learning rate for training model. This LR is internally scaled by num gpus during distributed training')
# parser.add_argument('--ft_lr', default=0.01, type=float, help='learning rate during fine-tuning iterations')
# parser.add_argument('--ft_epochs', default=2, type=int, help='number of fine-tuning epochs')

# args = parser.parse_args()
device_map = {'CPU': 'cpu', 'GPU': 'cuda'}

data_splits = get_data_splits_by_name(dataset_name='cifar100',
                                      model_name='resnet18',
                                      data_root='./Data',
                                      batch_size=16,
                                      num_workers=4,
                                      device=device_map['GPU'])

reference_model = get_model_by_name(model_name='resnet18',
                                    dataset_name='cifar100',
                                    pretrained=True,
                                    progress=True,
                                    device=device_map['GPU'])

config = {
    'deepsearch': True,
    'level': 1,
    'delta': 1,
    'device': 'GPU',
    'optimization': 'compression',
    'use_horovod': False,
    'export': {
        'format': ['onnx'],
        'root_path': "./Models",
        'kwargs': {'precision': 'fp16' if False else 'fp32'}
    },
    'full_trainer': {'optimizer': {'lr': 0.01}},
    'fine_tuner': {
        'loop_params': {
            'epochs': 2,
            'optimizer': {'lr': 0.01}
        }
    }
}

optimized_model = Neutrino(framework=TorchFramework(),
                           data=data_splits,
                           model=reference_model,
                           config=config).run(dryrun=True)

Files already downloaded and verified
Files already downloaded and verified
Loaded 122/122 modules
2023-01-09 08:33:14 - INFO: Starting job with ID: B5DEBCB7
2023-01-09 08:33:14 - INFO: Args: -f, /mnt/nfshome/risavsingh.saingar/.local/share/jupyter/runtime/kernel-927196e7-204f-4f91-9926-b6a75a11c718.json
2023-01-09 08:33:14 - INFO: 
+------------------------------------------------------------------------------------+
| Neutrino 5.3.3                                                 09/01/2023 08:33:14 |
+------------------------------------------------------------------------------------+
| License                                                    HVHEP-GCXFZ-KGUTB-TVGPI |
| Status                                                                       Valid |
| Expires                                                        2024-01-09 07:14:07 |
| Features                                                                           |
+-------------------------------------------------------

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
dataset_test = MNIST("./Data/", download=True, train=False, transform=transforms.ToTensor())
data_test = DataLoader(dataset_test, batch_size=8, shuffle=False, num_workers=16)

In [6]:
device = torch.device("cpu")
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices()[1], True)
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices()[2], True)
w = h = 464
class ModelTest1(nn.Module):
    def __init__(self):
        super(ModelTest1, self).__init__()
        self.conv1 = nn.Conv2d(1, 512, kernel_size=1, padding="same")
        self.conv2_1 = nn.Conv2d(512, 20, kernel_size=1, padding="same")
        self.conv2_2 = nn.Conv2d(20, 20, kernel_size=2, padding="same")
        self.conv2_ = nn.Conv2d(20, 512, kernel_size=1, padding="same")
    
    def forward(self, x):
        out = x
        out = self.conv1(out)
#         print(out.shape)
        out = self.conv2_1(out)
#         print(out.shape)
        out = self.conv2_2(out)
#         print(out.shape)
        out = self.conv2_(out)
#         print(out.shape)
        return out
    
class ModelTest2(nn.Module):
    def __init__(self):
        super(ModelTest2, self).__init__()
        self.conv1 = nn.Conv2d(1, 512, kernel_size=1, padding="same")
        self.conv2 = nn.Conv2d(512, 512, kernel_size=2, padding="same")
    
    def forward(self, x):
        out = x
        out = self.conv1(out)
#         print(out.shape)
        out = self.conv2(out)
#         print(out.shape)
        return out
    
m1 = ModelTest1().to(device)
m2 = ModelTest2().to(device)

from time import time
count = 0
t1 = 0
t2 = 0
start_ = time()
resize = tf.keras.layers.Resizing(w, h)
for d in tqdm(data_test):
    count += 1
    start = time()
    img, label = d
    img_ = img.to(device)
    img_ = img.permute(0, 2, 3, 1).cpu().numpy()
    img_ = torch.from_numpy(resize(img_).numpy()).permute(0, 3, 1, 2).to(device)
    end = time()
    t2 += end - start
#     start = time()
#     _ = m2(img_)
#     end = time()
#     t1 += end - start
    start = time()
    _ = m1(img_)
    end = time()
    t1 += end - start
    if count == 10:
        break
end = time()
print(t1, t2, end - start_)

  1%|▋                                                                                               | 9/1250 [00:08<18:51,  1.10it/s]

7.061826705932617 0.05769681930541992 8.207448244094849


In [ ]:
0.17606830596923828 1.9920427799224854 2.6187398433685303
0.017694950103759766 2.270150899887085 2.825688362121582

0.04861807823181152 37.70055794715881 38.31855249404907
0.20110821723937988 17.15498375892639 17.911318063735962
0.03659319877624512 16.53700351715088 17.155285596847534
0.49880242347717285 31.288609504699707 32.37795567512512

0.05565214157104492 30.639862775802612 31.320554733276367
0.0524594783782959 9.237820863723755 9.866660594940186

#CPU
14.253225803375244 0.04387092590332031 15.510392189025879

In [12]:
torch.cuda.empty_cache()

In [1]:
pwd

'/mnt/nfshome/risavsingh.saingar/Current/Edgification'